# Analysis of Terrorism Incidents

### a project of 

# Center of Data and Text Engineering and Mining (CoDTeEM)


## Terrorist Attacks in Pakistan (Yearly Data)

In [3]:
import numpy as np
import pandas as pd
import urllib
import time

from datetime import datetime, date

from IPython.display import display, HTML
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import math
# library for geo-json
from shapely.geometry import shape, Point, mapping

import ast
import string
import json

pd.options.mode.chained_assignment = None
mapbox_access_token = 'pk.eyJ1IjoibWFrcy1zaCIsImEiOiJjajRnMGhyNzcxZGFzMnd1ZnZteHF1YXo3In0.5R_XD1wl8F7ffCjAN1yxLg'


In [4]:
terror_data = pd.read_csv('data/globalterrorismdb_0718dist.csv', encoding='ISO-8859-1',
                          usecols=[0, 1, 2, 3, 8, 11, 12, 13, 14, 29, 35, 84, 98,99])
terror_data = terror_data.rename(
    columns={'eventid':'id', 'iyear':'year', 'imonth':'month', 'iday':'day',
             'country_txt':'country', 'provstate':'state', 'city':'city', 'targtype1_txt':'target',
             'weaptype1_txt': 'weapon', 'attacktype1_txt': 'attacktype', 'nkill':'nkill', 
             'nwound':'nwound', 'addnotes': 'info'})

terror_data['nkill'] = terror_data['nkill'].fillna(0).astype(int)
terror_data['nwound'] = terror_data['nwound'].fillna(0).astype(int)
terror_data = terror_data.dropna(how='any', subset=['latitude', 'longitude'])

In [5]:
terror_pak = terror_data[terror_data['country'] == 'Pakistan']
terror_india = terror_data[terror_data['country'] == 'India']
terror_afghan = terror_data[terror_data['country'] == 'Afghanistan']

#print('Pakistan size:', terror_pak.shape)
#print('India size:', terror_india.shape)
#print('Afghanistan size:', terror_afghan.shape)

terror_pak_assassination = terror_pak[terror_pak['attacktype'] == 'Assassination']
#print('Matrix size:', terror_pak_assassination.shape)
#terror_pak_assassination.head(5)

terror_pak.loc[:, 'day'] = terror_pak.apply(lambda row: str(row['id'])[6:8] if row['day'] == 0 else row['day'], axis=1)
#terror_pak.loc[:, 'date'] = pd.to_datetime(terror_pak[['day', 'month', 'year']])

terror_pak = terror_pak.drop_duplicates(['latitude', 'longitude', 'nkill'])
terror_peryear = np.asarray(terror_pak.groupby('year').year.count())
terror_years = np.arange(1970, 2016)
terror_years = np.delete(terror_years, [1])        

In [6]:
trace1 = go.Bar(
    x = terror_years,
    y = terror_peryear.cumsum(),
    name = 'Total number',
    marker=dict(
        color='#009999'
    )
)

trace2 = go.Scatter(
    x = terror_years,
    y = terror_peryear,
    name = 'Per year',
    mode = 'lines+markers',
    marker = dict(
        size = 5,
        symbol = 'diamond',
        color='#BF3030',
    ),
    line = dict(
        width = 2,
        color='#BF3030',
    ),
)

layout = go.Layout(
    title = 'Terrorist Attacks for years in Pakistan (1970-2016)',
    barmode='group',
    xaxis = dict(
        title = 'Year',
    ),
    yaxis = dict(
        title = 'Number of attacks',
    ),
    legend=dict(
        x=0,
        y=1
    )
)

data = [trace1, trace2]

fig = dict(data = data, layout = layout)
iplot(fig)